# Update sample prices data

In [64]:
import yfinance as yf

from pathlib import Path

ROOTDIR = Path.cwd().parent

In [65]:
from functools import lru_cache

INTERVAL = dict(daily="1d", hourly="1h", minute="1m")
MAXPERIOD = dict(daily="max", hourly="2Y", minute="7d")


@lru_cache
def get_prices(symbol: str, *, freq="daily", max_bars=0):
    interval = INTERVAL[freq]
    period = MAXPERIOD[freq]
    prices = yf.Ticker(symbol).history(interval=interval, period=period, auto_adjust=True)
    prices = prices.filter(["Open", "High", "Low", "Close", "Volume"])
    prices = prices.rename(columns=str.lower).rename_axis(index=str.lower)
    
    if freq == "daily":
        prices.index = prices.index.tz_localize(None)
    
    if max_bars > 0:
        prices = prices.tail(max_bars)
    return prices



In [66]:
SYMBOL = 'AAPL'
prices = get_prices(SYMBOL, freq="daily")

In [67]:
prices.index

DatetimeIndex(['1980-12-12', '1980-12-15', '1980-12-16', '1980-12-17',
               '1980-12-18', '1980-12-19', '1980-12-22', '1980-12-23',
               '1980-12-24', '1980-12-26',
               ...
               '2024-05-21', '2024-05-22', '2024-05-23', '2024-05-24',
               '2024-05-28', '2024-05-29', '2024-05-30', '2024-05-31',
               '2024-06-03', '2024-06-04'],
              dtype='datetime64[ns]', name='date', length=10960, freq=None)

In [68]:
for freq in INTERVAL.keys():
    prices = get_prices(SYMBOL, freq=freq, max_bars=5000)
    print(freq, len(prices), prices.index.dtype)


daily 5000 datetime64[ns]
hourly 3502 datetime64[ns, America/New_York]
minute 2728 datetime64[ns, America/New_York]


In [61]:
SAMPLES = ROOTDIR.joinpath("src/mplchart/samples")

for freq in INTERVAL.keys():
    fname = f"{freq}-prices.csv"
    prices = get_prices(SYMBOL, freq=freq)
    data = prices.to_csv(lineterminator="\n")
    outfile = SAMPLES.joinpath(fname)
    print(f"Updating {outfile.name} ...")
    outfile.write_text(data)

Updating daily-prices.csv ...
Updating hourly-prices.csv ...
Updating minute-prices.csv ...


In [63]:
from mplchart.samples import sample_prices

for freq in INTERVAL.keys():
    prices = sample_prices(freq=freq, max_bars=5000)
    print(freq, len(prices))


daily 5000
hourly 3501
minute 2670
